In [ ]:
'''
matlab commands



filename = "C:\Users\aaron\Desktop\Arbeit\SeabirdFileHandler\basic_emb.nc";

ncdisp(filename) # show header

prDM  = ncread(filename, "prDM");
t090C = ncread(filename, "t090C");
sal00 = ncread(filename, "sal00");


table(prDM(1:10), t090C(1:10), sal00(1:10))


# creates table with header and data

info = ncinfo(filename);

T = table();
for k = 1:length(info.Variables)
    name = info.Variables(k).Name;
    T.(name) = ncread(filename, name);
end

head(T,10)


# plot data

height(T)
head(T,10)
plot(T.t090C, T.prDM)
set(gca,"YDir","reverse")
'''

In [1]:
# pip install numpy netCDF4
import numpy as np
from netCDF4 import Dataset
import re

In [2]:
cnv_file = r"seabird_example_data\cnv\long_slow.cnv"
nc_file = "basic_emb.nc"

In [3]:
# read cnf
with open(cnv_file, "r", errors="ignore") as f:
    lines = f.readlines()

In [4]:
def clean_name(name):
    """netCDF-safe variablennames"""
    name = name.replace("/", "_")
    name = name.replace("-", "_")
    name = re.sub(r"[^a-zA-Z0-9_]", "", name)
    return name

In [5]:
# read variables
var_names = []
var_units = []

for line in lines:
    if line.startswith("# name"):
        name_part = line.split("=")[1]
        raw_name = name_part.split(":")[0].strip()
        clean_var = clean_name(raw_name)

        unit_match = re.search(r"\[(.*?)\]", line)
        unit = unit_match.group(1) if unit_match else ""

        var_names.append(clean_var)
        var_units.append(unit)

# find data starting point
data_start = None
for i, line in enumerate(lines):
    if line.strip() == "*END*":
        data_start = i + 1
        break

if data_start is None:
    raise RuntimeError("No *END* Marker")

# load measuringdata
data = np.loadtxt(lines[data_start:])

# write netcdf
nc = Dataset(nc_file, "w", format="NETCDF4")

# dimensions
nc.createDimension("obs", data.shape[0])

# variables
for i, name in enumerate(var_names):
    v = nc.createVariable(name, "f4", ("obs",))
    v[:] = data[:, i]
    if var_units[i]:
        v.units = var_units[i]

# safe CNV-Header
header_lines = []
for line in lines:
    if line.strip() == "*END*":
        break
    header_lines.append(line.rstrip())

nc.cnv_header = "\n".join(header_lines)

# globale attribute
nc.source = "Converted from Sea-Bird CNV"
nc.history = "CNV to netCDF conversion with Python"

nc.close()